# Lead Scoring Without CRM

This notebook demonstrates using everyrow's `rank()` utility to score investment firms by their likelihood to purchase research tools—without needing CRM data or prior interactions.

**Use Case:** A research tools company wants to rank investment firms by product fit. Traditional approaches either require expensive CRM integrations or burn credits on enrichment tools that provide data without interpretation.

**Why everyrow?** The `rank()` function can analyze public information (website descriptions, investment focus, team characteristics) and provide both a score AND reasoning for why a firm was scored a certain way.

In [1]:
import asyncio
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
from everyrow import create_session
from everyrow.ops import rank

## Load Investment Firm Data

In [2]:
firms_df = pd.read_csv("../data/investment_firms.csv")

print(f"Loaded {len(firms_df)} investment firms")
firms_df

Loaded 15 investment firms


,firm_name,aum_billions,strategy,description
0,Bridgewater Associates,120.0,"Global macro, systematic",World's largest hedge fund. Heavily systematic...
1,Pershing Square,12.0,Concentrated activist,"Bill Ackman's fund. Deep fundamental research,..."
2,Two Sigma,60.0,Quantitative,Tech-driven quantitative fund. Uses ML and big...
3,ValueAct Capital,16.0,Activist value,"Takes board seats, conducts operational deep d..."
4,Renaissance Technologies,55.0,Quantitative,Legendary quant fund. Medallion fund closed to...
5,Baupost Group,27.0,Value / distressed,"Seth Klarman's fund. Patient value investing, ..."
6,AQR Capital,98.0,Factor investing,Systematic factor strategies. Publishes academ...
7,Elliott Management,56.0,Activist / distressed,Paul Singer's fund. Legal and financial analys...
8,Point72,24.0,Multi-strategy,"Steve Cohen's fund. Hundreds of PMs, fundament..."
9,Lone Pine Capital,18.0,Long/short equity,"Tiger cub. Fundamental analysis, concentrated ..."


## Define Ranking Task

In [3]:
RANKING_TASK = """
Score each investment firm from 0-100 on their likelihood to PURCHASE THIRD-PARTY RESEARCH TOOLS.

HIGH likelihood (70-100) firms:
- Fundamental/discretionary strategies that require company analysis
- Activist investors who need deep research for campaigns
- Smaller teams that can't build everything in-house
- Known for research-intensive processes
- Short-sellers who need forensic analysis tools
- Firms that mention reading documents, research, or analysis

LOW likelihood (0-30) firms:
- Passive index funds (no stock picking = no research needs)
- Pure quantitative/systematic funds that build in-house
- Very large funds with unlimited internal resources
- Funds that explicitly mention building everything proprietary

MEDIUM (30-70) for firms with mixed signals.

Consider: Would this firm benefit from better research tools? Do they have the budget? 
Would they buy vs. build?
"""

## Run the Ranking

In [4]:
async def run_ranking():
    async with create_session(name="Research Tool Adoption Scoring") as session:
        print(f"Session URL: {session.get_url()}")
        
        result = await rank(
            session=session,
            task=RANKING_TASK,
            input=firms_df,
            field_name="score",
        )
        
        return result.data

results_df = await run_ranking()

Session URL: https://everyrow.io/sessions/461423af-82f0-477a-be26-36e5b1cc5012


## Analyze Results

In [5]:
# Sort by score
results_df = results_df.sort_values("score", ascending=False)

print(f"\n{'='*60}")
print("RESEARCH TOOL ADOPTION LIKELIHOOD RANKING")
print(f"{'='*60}\n")

for i, (_, row) in enumerate(results_df.iterrows(), 1):
    print(f"{i:2}. {row['firm_name'][:30]:30} | Score: {row['score']:3} | {row['strategy']}")
    if 'research' in row and pd.notna(row['research']):
        print(f"    Reasoning: {str(row['research'])[:70]}...")
    print()


RESEARCH TOOL ADOPTION LIKELIHOOD RANKING

 1. Muddy Waters Research          | Score:  98 | Short-selling / research
    Reasoning: {'score': "Muddy Waters Research, led by Carson Block, is a specialize...

 2. Pershing Square                | Score:  95 | Concentrated activist
    Reasoning: {'score': "Pershing Square is a concentrated activist fund led by Bill...

 3. Point72                        | Score:  95 | Multi-strategy
    Reasoning: {'score': "Point72 is a 'High likelihood' (70-100) candidate for purch...

 4. ValueAct Capital               | Score:  92 | Activist value
    Reasoning: {'score': 'ValueAct Capital is characterized by an extremely long-term...

 5. Baupost Group                  | Score:  92 | Value / distressed
    Reasoning: {'score': "Baupost Group is a premier fundamental value fund led by Se...

 6. Elliott Management             | Score:  92 | Activist / distressed
    Reasoning: {'score': "Elliott Management is a premier activist hedge fund with a ...

In [6]:
# Segment by tier
high_priority = results_df[results_df["score"] >= 70]
medium_priority = results_df[(results_df["score"] >= 40) & (results_df["score"] < 70)]
low_priority = results_df[results_df["score"] < 40]

print("\nSEGMENTATION:")
print(f"  High priority (70+):   {len(high_priority)} firms")
print(f"  Medium priority (40-69): {len(medium_priority)} firms")
print(f"  Low priority (<40):    {len(low_priority)} firms")


SEGMENTATION:
  High priority (70+):   9 firms
  Medium priority (40-69): 0 firms
  Low priority (<40):    6 firms


In [7]:
# Average score by strategy type
print("\nAVERAGE SCORE BY STRATEGY:")
print(results_df.groupby("strategy")["score"].mean().sort_values(ascending=False).to_string())


AVERAGE SCORE BY STRATEGY:
strategy
Short-selling / research    98.0
Concentrated activist       95.0
Multi-strategy              95.0
Activist / distressed       92.0
Activist value              92.0
Value / distressed          92.0
Value long/short            92.0
Event-driven activist       88.0
Long/short equity           85.0
Factor investing            25.0
Global macro, systematic    25.0
Passive / ETFs              15.0
Quantitative                15.0
Passive indexing            10.0


In [8]:
# Full results
results_df[["firm_name", "strategy", "aum_billions", "score", "research"]]

,firm_name,strategy,aum_billions,score,research
14,Muddy Waters Research,Short-selling / research,0.5,98,"{'score': 'Muddy Waters Research, led by Carso..."
12,Pershing Square,Concentrated activist,12.0,95,{'score': 'Pershing Square is a concentrated a...
13,Point72,Multi-strategy,24.0,95,{'score': 'Point72 is a 'High likelihood' (70-...
8,ValueAct Capital,Activist value,16.0,92,{'score': 'ValueAct Capital is characterized b...
9,Baupost Group,Value / distressed,27.0,92,{'score': 'Baupost Group is a premier fundamen...
10,Elliott Management,Activist / distressed,56.0,92,{'score': 'Elliott Management is a premier act...
11,Greenlight Capital,Value long/short,3.0,92,"{'score': 'Greenlight Capital, led by David Ei..."
7,Third Point,Event-driven activist,15.0,88,{'score': 'Third Point is a classic high-convi...
6,Lone Pine Capital,Long/short equity,18.0,85,{'score': 'Lone Pine Capital is a classic 'Tig...
4,Bridgewater Associates,"Global macro, systematic",120.0,25,{'score': 'Bridgewater is the world's largest ...
